In [13]:
import paramiko
import subprocess

ip = "172.16.16.24"
user = "root"
pw = "asd123!@"
cmd = ""
MK_PARTITION = ""
PARTITION = "/dev/sdb1"
MOUNT_DIR = "/mnt/test"
USER_NAME = "team1"

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd, get_pty=True)  # ← 여기가 핵심
    out = stdout.read().decode()
    err = stderr.read().decode()
    print(out if out else err)
    cli.close()

def pp(stdout):
    for i in stdout:
        print(i)

def install() :
    cc("apt -y install quota")
    cc("apt install -y expect")

def make_partition():
    global MK_PARTITION
    MK_PARTITION = input("무엇을 파티션할건지 적으세요 (ex. /dev/sda): ")
    script = f'''
#!/usr/bin/expect -f
spawn fdisk {MK_PARTITION}
expect "Command (m for help):"
send "n\\r"
expect "Select (default p):"
send "\\r"
expect "Partition number (1-4, default 1):"
send "\\r"
expect "First sector"
send "\\r"
expect "Last sector"
send "\\r"
expect "Command (m for help):"
send "w\\r"
expect eof
'''

    # 서버에 스크립트 파일 생성 후 실행
    cc(f"echo '{script}' > /tmp/mkpart.exp")
    cc("chmod +x /tmp/mkpart.exp")
    cc("expect /tmp/mkpart.exp")
    cc("lsblk")  # 파티션 생성 확인
    ## lsblk시 sda           8:0    0  102M  0 disk로 disk 로 떠야합니다


def format_partition():
    global PARTITION
    PARTITION = input("아까 만든 새 파티션을 입력하세요 (ex. /dev/sda1): ")
    cc(f'mkfs -t ext4 {PARTITION}')
    ## lsblk시 sda           8:0    0  102M  0 part로 part 로 떠야합니다

def add_user() :
    # 사용자 계정 생성
    global USER_NAME
    USER_NAME = input("생성할 사용자 계정명을 입력하세요 ")
    
    cmd = f"""
    # team1 계정이 존재하지 않으면 생성
    if ! id team1 &>/dev/null; then
        useradd -m -d /jupyter/{USER_NAME} {USER_NAME}
    else
        # 이미 존재하면 홈디렉토리만 확인 또는 새로 만듦
        mkdir -p /jupyter/{USER_NAME}
        usermod -d /jupyter/{USER_NAME} {USER_NAME}
    fi
    """
    cc(cmd)

# 비밀번호 설정 expect 스크립트
    script = f'''
    spawn passwd {USER_NAME}
    expect "New password:"
    send "asd123!@\\r"
    expect "Retype new password:"
    send "asd123!@\\r"
    expect eof
    '''
    
    cc("echo '{}' > /tmp/mkuser.exp".format(script))
    cc("chmod +x /tmp/mkuser.exp")
    cc("expect /tmp/mkuser.exp")

    
def mount() :
    global MOUNT_DIR
    MOUNT_DIR = input("마운트 할 경로를 입력하세요 (ex. /mnt/test): ")

    cmd = f"""
    sudo mkdir -p {MOUNT_DIR}
    sudo mount -t ext4 {PARTITION} {MOUNT_DIR}
    ls {MOUNT_DIR}
    """
    cc(cmd)

    # 명령 실행 후 출력받기
   # stdout = cc(cmd)
    # 결과를 Jupyter에서 바로 보기
 #   pp(stdout) # pp에서 에러 떠도 생기면 생긴겁니다. lost+found 터미널 들어가서서 생겻는지 직접확인하세요

def permanent_mount_config_remote():
    new_line = f"{PARTITION} {MOUNT_DIR} ext4    defaults,usrquota        0 0\n"

    fstab_contents = cc("cat /etc/fstab")  # 문자열 반환

    cc(f"echo '{new_line}' >> /etc/fstab")
    print("fstab에 새 항목 추가함:", new_line)
    cc("systemctl daemon-reload")
    cc(f"mount -o remount {MOUNT_DIR}")
    
def diskquota() :
    # 파일 시스템 언마운트 후에 ext4 내장 쿼터 기능 활성화
    # 이부분은 루트권한이 필요합니다.. 안될시에 터미널에 직접 입력하세요.
    cmd = f"""
    sudo bash -lc '
    umount {MOUNT_DIR}
    tune2fs -O quota {PARTITION}
    mount -a
    quotacheck -cu {MOUNT_DIR}
    quotaon {MOUNT_DIR}
    '
    """
    cc(cmd)
def restrict_disk_and_check() :
    cc(f""" 
    sudo bash -lc '
    #edquota -u team1  <- 이건 편집하는 기능이라..
    quotaon -u {MOUNT_DIR}
    repquota {MOUNT_DIR}
    '
    """) 

    
def main():
#    install()
   #==================
#    make_partition()
#    format_partition()
#    add_user()
   #===================
#    mount() 
#    permanent_mount_config_remote()
    diskquota()
    restrict_disk_and_check()
    
main()

umount: /mnt/test: target is busy.
tune2fs 1.47.2 (1-Jan-2025)
The quota feature may only be changed when the filesystem is unmounted and not in use.
mount: ext4: mount point does not exist.
       dmesg(1) may have more information after failed mount system call.
quotacheck: Your kernel probably supports ext4 quota feature but you are using external quota files. Please switch your filesystem to use ext4 quota feature as external quota files on ext4 are deprecated. You can enable the feature by unmounting the file system and running 'tune2fs -O quota <device>'.
quotaon: Your kernel probably supports ext4 quota feature but you are using external quota files. Please switch your filesystem to use ext4 quota feature as external quota files on ext4 are deprecated. You can enable the feature by unmounting the file system and running 'tune2fs -O quota <device>'.

quotaon: Your kernel probably supports ext4 quota feature but you are using external quota files. Please switch your filesystem to 